In [1]:
import joblib
import cv2
import matplotlib.pyplot as plt
import pywt
import numpy as np

In [2]:
md = joblib.load("saved_model.pkl")

In [3]:
path = './image3.jpg'
img = cv2.imread(path)

In [4]:
face_cascade_path = './opencv-master/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml'
face_cascade_path = './opencv-master/opencv-master/data/haarcascades/haarcascade_eye.xml'

In [5]:
face_cascade = cv2.CascadeClassifier(face_cascade_path)
eye_cascade = cv2.CascadeClassifier(face_cascade_path)

In [6]:
def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [7]:
im_har = w2d(img,'db1',5)

In [8]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    if(img is not None):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                return roi_color

In [9]:
cropped_image = get_cropped_image_if_2_eyes(path)

In [10]:
scalled_raw_img = cv2.resize(cropped_image, (32, 32))
img_har = w2d(cropped_image,'db1',5)
scalled_img_har = cv2.resize(img_har, (32, 32))
combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))

In [11]:
X = []
X.append(combined_img)
X = np.array(X).reshape(len(X),4096).astype(float)

In [12]:
ans_dict = {'serena_williams': 0,
 'maria_sharapova': 1,
 'roger_federer': 2,
 'virat_kohli': 3,
 'lionel_messi': 4}

In [13]:
for key,value in ans_dict.items():
    if(md.predict(X)==value):
        print(key)
        break

maria_sharapova
